# Welcome to medicinal chemistry workshop!

### Story 
_You are a chemist working in the laboratory of a large pharmaceutical company. **Your task is to study drug compounds and compare their properties to those of aspirin.** Water solubility is particularly important, as the drug compound needs to dissolve well in blood. The biological effect and metabolism of the drug compound can be modeled later._

_You have already studied many different promising drug compounds without success in the laboratory using demanding solubility tests that depend on the acidity and temperature of the solutions. The project deadline for completing the solubility tests is approaching. Therefore, new ways must be found to determine the best possible drug compounds from the sample for further research and experimentation._

_Fortunately, you have access to various laboratory results and modeling tools that pharmaceutical and chemical software developers have been working on for decades. It's time to dive into the world of cheminformatics and solve the water solubility tests before the deadline!_


## Technical instructions
This work is done in a Jupyter notebook. This notebook is an environment that contains cells that make it **modular**. This modularity means that the computer reads each cell individually, but in top-down order. The cells can be divided into two types in this work instruction:

- Code cells, in which commands can be written. Often, before this, they load various libraries that are used to execute commands and processes.
- Markdown language cells, which contain text that can be formatted like in a text editor, but with certain **commands** using different characters either before or on both sides of a word.

You can edit all cells by double-clicking on the cell and writing commands on the lines or pasting copied text into them with the command. You can run each cell and see the effect of your changes by pressing: **CTRL + ENTER**

If you encounter technical problems (for example, a cell does not work), this may be because you have run the cell in a different order than normal. Try to run the cells from top to bottom. If this is not the cause of the problem, use the save command and then press **CTRL + SHIFT + R**. This will reset the notebook. Alternatively, you can run the 'Restart the kernel' command from the top bar.

---

# PART I

### Kertaus SMILES-säännöistä

**Taulukko 1.** SMILES-säännöt erilaisilla esimerkeillä.
| Symboli | Esim. | Selitys |
| --- | --- | --- |
| Kemiallinen merkki | `O` | Kyseessä on molekyyli, vesi. Alkuaineet ilmaistaan kemiallisina merkkeinä, kuten jaksollisessa järjestelmässä. Vetyjä ei ilmoiteta vaan ne tulkitaan automaattisesti osaksi yhdistettä perustuen alkuaineen tyypilliseen valenssiin, eli sidosten lukumäärään. Katso koodista miten ne saadaan näkyviin. |
| Kovalenttinen sidos | `C=C` | Yksinkertaisia sidoksia ei esitetä, mutta kaksois- ja kolmoissidokset esitetään = ja # symbolein. |
| Vedyt | `C` | Merkitään vain kiraalisuuden määrittämisessä, C vastaa metaania eli CH4:stä. |
| Haarat | `CCCC(C)CC` | Sulut () ilmoittavat haaran ja suluissa ilmoitetaan mikä haara on kyseessä.  |
| Rengasrakenne | `C1CCCCC1` tai `C1CCC(C(C1)Cl)Cl` | Numerot 1-9 ilmoittavat atomin sijainnin rengasrakenteessa. Vedyt korvanneet substituentit voidaan laittaa sulkuihin. HUOM. Cl on kloori-atomi.|
| Aromaattisuus | `c1ccccc1` | Rengasrakenteen kiraalisuus voidaan esittää vaihtamalla kirjaimet pieniksi. |
| Kiraalisuus | `C[C@@H](C(=O)O)N`  | L-alaniini, jossa @@ hiilen jälkeen osoittaa sen kiraaliseksi. |
| Isotoopit | `[13C]` | C13-isotoopeista koostuva etaanimolekyyli. Numerot ennen alkuainemerkkiä osoittavat isotoopin. |
| Sähköinen varaus ja radikaalit | `[NH4+].[Cl-]` tai `C[CH2].[CH3]`  | Varaus + tai -, ja radikaalit usein luetaan erityisillä algoritmeilla ohjelmistoissa, jotka tulkitsevat SMILES-merkkijonoja. Niille ei ole vakiintunutta merkintää.  |

### 1. Piirretään yhdisteitä SMILES-merkkijonojen avulla

In [ ]:
# Alla olevan koodin avulla voit piirtää yhdisteen syöttämällä SMILES-merkkijonon kohtaan
# smiles = ' ' kohtaan ja ajamalla solu painamalla CTRL + Enter.
# Aloita kokeilemalla joku SMILES-merkkijono yllä olevasta taulukosta.

# Tuodaan tarvittavat moduulit kirjastoista.
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

smiles = 'C'  # Siirrä haluamasi SMILES-merkkijono ' ' merkkien väliin C:n tilalle. Paina lopuksi CTRL + ENTER, jolloin ajat solun.
m = Chem.MolFromSmiles(smiles)

# m = Chem.AddHs(m) # Voit tällä komennolla visualisoida kaikki vedyt, jos haluat. Poista ensimmäinen #-merkki aktivoidaksesi asetuksen

# Luodaan funktio moltosvg, joka luo SMILES-merkkijonosta rakennekaavan kuvan SVG-muotoon.
def moltosvg(mol, molSize=(350, 300), kekulize=True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0], molSize[1])
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    svg = drawer.GetDrawingText()
    return SVG(svg)
    

rakenneKaava = moltosvg(m)  # Määritetään rakenneKaava muuttuja ja asetetaan sen arvoksi SVG kuva moltosvg funktiolla.
display(rakenneKaava)       # Tulostetaan rakennekaavan kuva.

###  Kysymys 2.
**Minkä yhdisteen piirsit ja mitä funktionaalisia ryhmiä yhdisteellä on?**

Näihin kysymyksiin vastataan Microsoft Forms -lomakkeelle.

---

## 2. Luodaan liitostaulukko
Testataan miten liitostaulukot toimivat. Valitse jokin monimutkaisempi SMILES ja kokeile luoda liitostaulukko sille seuraavan koodin avulla. Voit myös kokeilla useammalle SMILESille. Tietokone aloittaa indeksoinnin nollasta, eli ensimmäinen hiili on taulukossa ja rakenne kuvassa numerolla 0.

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
import numpy as np
import pandas as pd

mol = Chem.MolFromSmiles('C') # Vaihda tähän jokin monimutkainen SMILES C:n tilalle.

# Määritellään asetuksia rdkitin piirtofunktiolle.
d2d = rdMolDraw2D.MolDraw2DSVG(300,300)
d2d.drawOptions().addAtomIndices = True  # Lisätään kuvaan hiilten numerointi. Numerointi alkaa tietokoneelle numerosta 0.
d2d.DrawMolecules([mol])
d2d.FinishDrawing()
svg = SVG(d2d.GetDrawingText())

num_atoms = mol.GetNumAtoms()
connectivity_matrix = np.zeros((num_atoms, num_atoms), dtype="f")
pd.options.display.float_format = lambda x: ('%g' % x)


# Luodaan liitostaulukko for-silmukan avulla.
for bond in mol.GetBonds():
    begin_atom_idx = bond.GetBeginAtomIdx()    
    end_atom_idx = bond.GetEndAtomIdx()
    bond_type = (bond.GetBondTypeAsDouble())    
    connectivity_matrix[begin_atom_idx][end_atom_idx] = bond_type
    connectivity_matrix[end_atom_idx][begin_atom_idx] = bond_type
    
df = pd.DataFrame(connectivity_matrix)

# Tulostetaan rakennekaava ja liitostaulukko.
display(svg, df)

### Kysymys 3.
**Mikä sidos on toisen ja kolmannen hiilen välillä?**

----

## 3. Tutustutaan stereoisomeriaan

In [ ]:
# Tällä koodilla voit piirtää streokemiallisia yhdisteitä
# Aluksi mol1 ja mol2 määritellään SMILES:ksi

mol1 = Chem.MolFromSmiles('C/C=C/C') # Huomaa, että tämä toimii vain isomeerisille SMILES-merkkijonoille
mol2 = Chem.MolFromSmiles('C[C@H](O)[C@@H](N)C(O)=O')


d2d = rdMolDraw2D.MolDraw2DSVG(600,280,300,280)
d2d.drawOptions().addStereoAnnotation=True # Komento merkitsee piirrokseen kiraaliset keskukset.
d2d.DrawMolecules([mol1,mol2]) # RDKit pystyy tulostamaan vain kaksi molekyyliä kerrallaan
d2d.FinishDrawing()
rakenneKaavat = SVG(d2d.GetDrawingText())
display(rakenneKaavat)

### Kysymys 4.
**Onko vasemmanpuoleinen molekyyli cis vai transmuotoinen? Montako kiraliakeskusta löytyy oikeanpuoleisesta yhdisteestä?**

---

## 4. Tutustutaan PUG-REST -menetelmään ja SMILES:hin

Ohjelmistoympäristöön voi tuoda helposti yhdisteitä teettämällä kyselyitä PubChem-palvelimelta. Tämä on erityisen hyödyllistä, kun halutaan nopeasti tuoda tietoa lääkeyhdisteistä ilman hakukoneita luettelemalla haluttuja molekyylejä osaksi koodia. 

Tästä on erityistä hyötyä, kun haluat hakea SMILES-merkkijonoja tietokannasta.

### Esimerkkejä lääkkeistä
| Lääkeaine                  | Tuotenimi                                                                                                                    | Kuvaus |
| ----------- | -------------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| Aspirin                | (Aspirin)                                           | Maailman käytetyin tulehduskipulääke. Ehkäisee veren hyytymistä, joten käytetään sydäninfarktien ensihoitoon.                                                                                                                                               |
| Ibuprofen              | (Burana)                                             | Tulehduskipulääke.                                                                                                                                                                                                                                       |
| Paracetamol            | (Panadol)                                            | Kipulääke.                                                                                                                                                                                                                                               |
| Morphine                | (Morfiini)                                           | Voimakas kipulääke. Käytetään sairaalahoidossa.                                                                                                                                                                                                          |
| Cortisone	             | (Prednison, Hydrocortison)                           | Kortisonia käytetään esim. astmaan, allergiohin ja ihosairauksien hoitoon. Käytetään hengitysteitse, tablettina sekä voiteena.                                                                                                                             |
| Penicillin V           | (V-Pen)                                              | Antibiotti bakteeri-infektioiden hoitoon.                                                                                                                                                                                                                |
| Cetirizine	            | (Heinix)                                             | Antihistamiini allegioiden hoitoon.                                                                                                                                                                                                                      |
| Insulin                | (Lantus)                                             | Hormoni diabeteksen hoitoon.                                                                                                                                                                                                                             |
| Paclitaxel             | (Taxol)                                              | Syöpälääke.                                                                                                                                                                                                                                              |
| Sertraline              | (Zoloft)                                             | Masennuslääke, käytetään myös paniikkihäiriöön ja pakkoireisiin.                                                                                                                                                                                         |


Valitse kolme tutkittavaa lääkeainetta. Voit hyödyntää yllä olevaa taulukkoa, tai käyttää itse valitsemiasi lääkeaineiden nimiä. Syötä lääkaineet koodiin englanniksi! Voit etsiä netistä lääkkeen vaikuttavan aineen nimen. Älä käytä lääkkeen tuotenimeä.

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
import requests

# Luodaan funktio, jolla haetaan PubChemin tietokannasta tiedot käyttäen PUG-REST menetelmää.
# Älä muokkaa tätä osuutta koodista!
def retrieve_names(compound_names):
    pugrest = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    pugoper = "property/CanonicalSMILES"
    pugout = "txt"
    results = {}
    for compound_name in compound_names:
        pugin = "compound/name/" + compound_name
        url = f"{pugrest}/{pugin}/{pugoper}/{pugout}"
        response = requests.get(url)
        if response.status_code == 200:
            results[compound_name] = response.text.strip()  # Poistetaan ylimääräiset välilyönnit.
        else:
            results[compound_name] = f"Error: {response.status_code}"
    return results

# Muokkaa alla olevaa koodiriviä. Lisää tähän valitsemasi englanninkieliset lääkeaineiden nimet lainausmerkkien sisään.
compound_names = ["", "", ""] 
# Muokkaa vain yllä olevaa koodiriviä.

# Haetaan retrive_names funktiota käyttäen SMILES-merkkinojot.
canonical_smiles = retrieve_names(compound_names)

# Tulostetaan tulokset.
for compound_name, smiles in canonical_smiles.items():
    print(f"Canonical SMILES for {compound_name}: {smiles}")

### Kysymys 5.
**Mitä lääkeaineita valitsit ja mitkä ovat niiden SMILES-merkkijonot?**

---

## 5. Piirretään lääkeaineita

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
from rdkit import Chem 
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import rdDepictor 
from rdkit.Chem.Draw import rdMolDraw2D 
from IPython.display import SVG 

smiles = '' # Testaa hakujesi tuloksia lisäämällä SMILES kerrallaan '' merkkien väliin. 
m = Chem.MolFromSmiles(smiles)
# m = Chem.AddHs(m) # Voit tällä komennolla visualisoida kaikki vedyt, jos haluat. Poista ensimmäinen #-merkki aktivoidaksesi asetuksen


# Luodaan funktio moltosvg_with_indices, joka luo SMILES-merkkijonosta rakennekaavan kuvan SVG-muotoon.
def moltosvg_with_indices(mol, molSize = (350,300), kekulize = True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer = Chem.Draw.rdMolDraw2D.MolDraw2DSVG(molSize[0], molSize[1])
    drawer.drawOptions().addAtomIndices = True
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg)
    

rakenneKaava = moltosvg_with_indices(m)  # Määritetään rakenneKaava muuttuja ja asetetaan sen arvoksi SVG kuva moltosvg_with_indices funktiolla.
display(rakenneKaava)       # Tulostetaan rakennekaavan kuva.

### Kysymys 6.
**Mitä funktionaalisia ryhmiä löydät lääkeaineista?**

---

## 6. Tutustutaan IUPAC-nimiin ja PubChem tietokantaan

In [ ]:
# Tällaisella koodilla voit hakea erilaisilla triviaali- tai IUPAC-nimillä tiettyjä tietoja PubChem-tietokannasta.

# Tuodaan tarvittavat moduulit kirjastoista.
import pubchempy as pcp 
import pandas as pd
pd.options.display.max_colwidth = 1000 # Määritetään pandas-kirjaston sarakkeille oletusarvoa pidempi maksimi pituus.

# Lista esimerkki aineista. Voit lisätä tähän haluamasi yhdisteet.
names = ["water", 
         "benzene", 
         "methanol", 
         "ethene", 
         "ethanol", 
         "propene", 
         "1-propanol", 
         "2-propanol", 
         "butadiene", 
         "1-butanol", 
         "2-butanol", 
         "tert-butanol"]

# Käytetään for-silmukkaa listan läpikäymiseen ja tallennenetaan aineiden ID:t listaan.
# PubChem-tietokannan yhdisteet on indeksoitu (ID) ja CID on lyhenne kemialliselle indeksille.
# Ne ovat uniikkeja nimikkeille ja toimivat selkeämpänä perustana haulle SMILES:n tai muiden tietojen sijaan.
compound_ids = []
for name in names:
    try:
        cp = pcp.get_compounds(name, namespace='name')
        if cp:
            compound_ids.append(cp[0].cid)
    except Exception as e:
        print(f"Error retrieving CID for '{name}', : {e}")
results = pcp.get_properties(['IUPACName', 'MolecularFormula', 'MolecularWeight', 'CanonicalSMILES'], compound_ids)

# Luodaan tuloksista pandas-kirjaston taulukko ja tulostetaan se.
df = pd.DataFrame(results)
display(df)

### Kysymys 7.
**Miten valitsemasi lääkeaineen nimi eroaa IUPAC-nimestä?**

---

# OSA II

QSAR-algoritmiä voidaan hyödyntää lääkeaineiden biologisen aktiivisuuden ennustamiseen. Ihan ensimmäiseksi algoritmia hyödynnettiin silti kiehumispisteen ennustamiseen molekyylirakenteen perusteella. Siksi aloitetaan tässä työpajassa visualisoimalla moolimassan ja kiehumispisteen korrelaatiota. 

### 7. Tutkitaan mooolimassan ja kiehumispisteen korrelaatiota

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
import pandas as pd

pd.options.display.max_colwidth = 1000  # Määritetään pandas kirjaston sarakkeille oletusarvoa pidempi maksimipituus.
df = pd.read_csv('../../data/BP.csv')              # Luetaan tiedosto taulukoksi pandas-kirjaston avulla.
display(df)                             # tulostaa taulukon, koska määritelty df tarkoittaa dataframe -komentoa.

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
import matplotlib.pyplot as plt

plt.scatter(df.Moolimassa, df.Kiehumispiste_K)      # Tekee verrannon kiehumispisteen (K) suhteesta moolimassaan
plt.xlabel('Moolimassa')                            # Asetetaa x-akselille nimi
plt.ylabel('Kiehumispiste')                         # Asetetaa y-akselille nimi
plt.show()                                          # Tulostetaan kuvaaja

### Kysymys 8.
**Mitä voit päätellä molekyylien kiehumispisteistä ja niiden suhteesta moolimassaan? Miksi sama moolimassa voi johtaa eri kiehumispisteeseen?**

---

### 8. Lineaarisen regressiomallin sovittaminen

Lineeariset sovitukset ovat tuttuja varmasti matematiikan tunneilta. Tämä matemaattinen mallintaminen on yksi keskeisimpiä menetelmiä keminformatiikassa, koska sillä voidaan tarkastella erilaisten muuttujien suhdetta toisiinsa (korrelaatio) alkaen kahdesta tai usemmasta tekijästä. Alla on koodia, jolla voidaan sovittaa regressiomalli datasettiin. Mitä huomaat, kun koodi tulostaa graafin? 

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Tuodaan dataa .csv tiedostosta taulukoksi
df = pd.read_csv('../../data/BP.csv')

# Sovitetaan lieneaarinen regressiomalli
model = LinearRegression()
X = df[['Moolimassa']]          # Riippumaton muuttuja
y = df[['Kiehumispiste_K']]     # Riippuva muuttuja
model.fit(X, y)

# Piirrä  pistekuvio datasta 
plt.figure()
sns.scatterplot(x='Moolimassa', y='Kiehumispiste_K', data=df, color='tab:blue', label='Datapisteet')

# Piirrä regressiosuora
plt.plot(X, model.predict(X), color='tab:red', label='Lineaarinen regressio')

# Aseta otsikot ja mitat
plt.xlabel('Moolimassa')
plt.ylabel('Kiehumispiste (K)')
plt.title('Lineaarinen regressiomalli')

# Näytä selite
plt.legend()

# Näytä kuvaaja
plt.show()

### Kysymys 9.
**Seuraako punainen regressiosuora datapisteitä tarkasti?**

---

### 9. Lääkeaineiden toimivuuden ennustaminen
#### Tehtäväsi

**Löytää yhdisteitä, joilla on aspiriiniin verrattavissa oleva liukoisuus LogS-arvo -2.1 tutkimalla niitä alla sijaitsevan koneoppimisen mallin avulla.** LogS-arvo kuvaa yhdisteen vesiliukoisuutta ja alla olevassa taulukossa on kerrottu siitä tarkemmin. 

| Liukoisuus (LogS) | LogS < -4            | -4 < LogS < -2            | -2 < LogS < 0 |
| ----------------- | ----------------- | -------------------- | ------------------------- |
| Tulkinta                           | Ei liukene veteen                  | Liukenee melko hyvin                     | Liuekenee erinomaisesti                            |
| Pitoisuus veressä                  | Minimaalinen                       | Hyvä                                     | Korkea, pitoisuuden nousu                          |


On aika tutkia vähintään 5 yhdistettä aloittamalla niiden SMILES-nimistä. Käytä aina valitsemiesi yhdisteiden lisäksi **aspiriiniä** referenssinä:

- Voit etsiä tietokantoja joko vapaasti tai PUG-REST -komennoilla
- Hyödynnä kokeellista dataa yhdisteistä (taulukko).

Arvioi niiden soveltuvuutta lääkeyhdisteiksi erityisesti **liukoisuuden arvon ja Lipinskin viiden säännön suhteen**.

**Tämän työn tarkoitus on kartoittaa pieni otos samaa liukoisuutta omaavista yhdisteistä**, joita esimerkiksi sinä tai muut voivat **hyödyntää lähtökohtana seuraaville keminformatiikan liukoisuusmalleille**. Kaikki lähtee tunnetuista ja varmennetuista datapisteistä! Onnea matkaan!

In [ ]:
# Voit esimerkiksi tutkia vastaako mallin arvot kokeellisten tutkimusten tuloksia. 
# Datasetissä on epäonnistuneiden ennusteiden ja oikeiden ennusteiden tuloksia.

# Tuodaan tarvittavat moduulit kirjastoista.
import numpy as np
import pandas as pd

# Määriteään pandaskirjaston tulostusasetuksia
pd.options.display.max_colwidth = 1000  # Määritetään pandas kirjaston sarakkeille oletusarvoa pidempi maksimipituus.
pd.options.display.max_rows = 500       # Määritetään pandas kirjastolle oletusarvoa enemmän rivejä.

df = pd.read_csv('../../data/kokeellisetLogS_referenssipisteet.csv') # Luetaan tiedosto taulukoksi pandas -ohjelmiston avulla.
df = df.loc[:100, ['Chemical name', 'LogS exp (mol/L)', 'Test', 'SMILES']] # Valitaan tulukosta haluamamme sarakkeet

# Tulostetaan taulukko
display(df)

### 10. Koneoppimisen malli

In [ ]:
# Tuodaan koneoppimisen malli lääkeyhdisteiden vesiliukoisuuden ennustamiseksi. 

# Tuodaan tarvittavat moduulit kirjastoista.
from pycaret.classification import *

# Ladataan ennanltaopetettu koneoppimisen malli.
model = load_model("../../data/WaterSoulubility_02_06_2025_model")

Käytä tätä koneoppimisen mallia ennustamaan yhdisteiden liukoisuutta ja soveltuvuutta lääkeaineiksi! Syötä SMILES-jonoja yhdisteille mitä haluat tutkia, mutta muista erottaa ne pilkulla ja rivinvaihdolla.

In [ ]:
# Tuodaan tarvittavat moduulit kirjastoista.
from chem_util import descriptors_from_smiles 

# Syötä listaan SMILES-merkkijonoja käyttäen lainausmerkkejä jokaisen yhdisteen ympärille esimerkin mukaisesti.
# Esimerkiksi seuraavat yhdisteet.
smiles = [
  "C1CCCCC1",
  "CC(C)Cc1ccc(cc1)C(C)C(=O)O",
  "CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=C(C=C3)O)N)C(=O)O)C",
  "CC1=C(C(=O)C(=C(C1=O)OC)OC)CC=C(C)CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CCC=C(C)C", 
  "F[C@H]1CC[C@H](O)CC1", 
  "C1CC1[C@H](F)C1CCC1", 
  "CC(=O)OC1=CC=CC=C1C(=O)O"] # Tämä on Aspiriini!

x = descriptors_from_smiles(smiles) # Koneoppimisen malli hyödyntää molekulaarisia deskriptoreita laskiessaan kysyttyjä arvoja SMILES:ien perusteella
display(x) # Tulostetaan taulukko.

### 11. Ennustetaan liukoisuutta

In [ ]:
# Ennustetaan käyttäen koneoppimisen mallia.
ennuste = model.predict(x)

# Luodaan taulukko ennuisteista ja niiden SMILES:sseista ja tulostetaan se.
df = pd.DataFrame({"Ennustettu LogS": ennuste, "SMILE": smiles})
display(df)

### Kysymys 10.
**Mitä yhdisteitä tutkit ja liukenevatko ne veteen?**

### Kysymys 11.
**Miten valisemasi yhdisteen liukoisuus eroaa aspiriinista, joka tunnetusti liukenee hyvin veteen ja sopii lääkkeenä suun kautta annettavaksi?**

---

### 12. SwissADME
SwissADME on ilmainen verkkotyökalu, jota käytetään arvioimaan kemiallisten yhdisteiden farmakokineettisiä ominaisuuksia. Se auttaa ennustamaan yhdisteiden imeytymistä, jakautumista, metaboliaa ja erittymistä, sekä analysoimaan niiden kemiallista reaktiivisuutta ja toksisuutta. Työkalu on erityisen hyödyllinen lääkekehityksessä, koska se tarjoaa nopean ja helpon tavan arvioida yhdisteiden potentiaalia toimia tehokkaina ja turvallisina lääkkeinä.

http://www.swissadme.ch/ 

### Kysymys 12.
**Tutki samoja yhdisteitä SwissADMEn avulla. Saatko saman ennusteen yhdisteen liukoisuudesta ja toimivuudesta lääkkeenä SwissADMENn avulla verrattuna yllä olevaan koneoppimismalliin?**

---